https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby

In [1]:
import numpy as np
import pandas as pd 

In [2]:
keys = np.array([
        ['A', 'B'],
        ['A', 'B'],
        ['A', 'B'],
        ['A', 'BB'],
        ['C', 'D'],
        ['C', 'D'],
        ['C', 'DD'],
        ['E', 'F'],
        ['E', 'FF'],
        ['G', 'H'] 
        ])

df = pd.DataFrame(
    np.hstack([keys,np.random.randn(10,4).round(2)]), 
    columns = ['col1', 'col2', 'col3', 'col4', 'col5', 'col6']
)

df[['col3', 'col4', 'col5', 'col6']] = \
    df[['col3', 'col4', 'col5', 'col6']].astype(float)

In [3]:
df

,col1,col2,col3,col4,col5,col6
0,A,B,1.33,0.86,-2.69,0.56
1,A,B,1.34,-1.06,-1.56,-0.12
2,A,B,-0.36,-0.23,1.71,-0.49
3,A,BB,-1.05,-0.27,-0.96,0.52
4,C,D,1.24,-0.07,-1.53,1.41
5,C,D,2.12,-0.98,0.29,2.25
6,C,DD,0.74,1.11,1.59,1.53
7,E,F,-0.53,-1.21,0.21,-0.28
8,E,FF,-0.27,0.13,-0.68,-0.31
9,G,H,1.00,0.16,0.66,-0.35


In [4]:
df.groupby(['col1', 'col2']).size().reset_index(name='counts')

,col1,col2,counts
0,A,B,3
1,A,BB,1
2,C,D,2
3,C,DD,1
4,E,F,1
5,E,FF,1
6,G,H,1


In [5]:
(df.groupby(['col1', 'col2'])
 .agg({
    'col3': ['mean', 'count'], 
    'col4': ['median', 'min', 'count']
}))

col3         col4            
           mean count median   min count
col1 col2                               
A    B     0.77     3 -0.230 -1.06     3
     BB   -1.05     1 -0.270 -0.27     1
C    D     1.68     2 -0.525 -0.98     2
     DD    0.74     1  1.110  1.11     1
E    F    -0.53     1 -1.210 -1.21     1
     FF   -0.27     1  0.130  0.13     1
G    H     1.00     1  0.160  0.16     1

In [6]:
gb = df.groupby(['col1', 'col2'])
counts = gb.size().to_frame(name='counts')
(counts
 .join(gb.agg({'col3': 'mean'}).rename(columns={'col3': 'col3_mean'}))
 .join(gb.agg({'col4': 'median'}).rename(columns={'col4': 'col4_median'}))
 .join(gb.agg({'col4': 'min'}).rename(columns={'col4': 'col4_min'}))
 .reset_index()
)

,col1,col2,counts,col3_mean,col4_median,col4_min
0,A,B,3,0.77,-0.230,-1.06
1,A,BB,1,-1.05,-0.270,-0.27
2,C,D,2,1.68,-0.525,-0.98
3,C,DD,1,0.74,1.110,1.11
4,E,F,1,-0.53,-1.210,-1.21
5,E,FF,1,-0.27,0.130,0.13
6,G,H,1,1.00,0.160,0.16


In [59]:
my_groupby = ['col1', 'col2']
df1 = (df.groupby(my_groupby)
 .agg({
    'col3': ['mean', 'count'], 
    'col4': ['median', 'min', 'count']
}))
df1.columns = df1.columns.map('_'.join).str.strip('_')
df1.index = df1.index.map('|'.join).str.strip('|')
df1.reset_index(inplace=True)
df1[my_groupby] = df1['index'].str.split('|', n=1, expand=True)
df1.drop(columns = 'index', inplace=True)
df1 = df1.iloc[:,[-x for x in range(len(my_groupby),-1,-1)]+list(range(len(df1.columns)-len(my_groupby)))]
df1

,col1,col2,col3_mean,col3_mean,col3_count,col4_median,col4_min,col4_count
0,A,B,0.77,0.77,3,-0.230,-1.06,3
1,A,BB,-1.05,-1.05,1,-0.270,-0.27,1
2,C,D,1.68,1.68,2,-0.525,-0.98,2
3,C,DD,0.74,0.74,1,1.110,1.11,1
4,E,F,-0.53,-0.53,1,-1.210,-1.21,1
5,E,FF,-0.27,-0.27,1,0.130,0.13,1
6,G,H,1.00,1.00,1,0.160,0.16,1


In [60]:
list(map('|'.join, [('A', 'B'),'BC']))

['A|B', 'B|C']

In [61]:
df[['col1', 'col2']].groupby('col1').agg({'col2':'unique'})

,col2
col1,
A,"[B, BB]"
C,"[D, DD]"
E,"[F, FF]"
G,[H]


In [62]:
# to jsem chtěl
mux = pd.MultiIndex.from_tuples([('A','B'), ('C','D')], 
                                 names=[None, 'date'])
test_df = pd.DataFrame({'col1': 125, 'col2': 225} , index = mux)
test_df

,,col1,col2
,date,,
A,B,125,225
C,D,125,225


In [63]:
test_df.index

MultiIndex([('A', 'B'),
            ('C', 'D')],
           names=[None, 'date'])

In [64]:
test_df.index.values

array([('A', 'B'), ('C', 'D')], dtype=object)

In [65]:
test_df.index.names

FrozenList([None, 'date'])

In [66]:
test_df.values

array([[125, 225],
       [125, 225]])

In [67]:
test_df.groupby('col1').size().reset_index(name='counts')

,col1,counts
0,125,2


In [68]:
test_df.groupby(['col1', 'col2']).size().reset_index(name='counts')

,col1,col2,counts
0,125,225,2
